In [1]:
#### Get Road Distance from BaiduMap API ####
# Programmer : Dan Qin
# Date       : 2018.06.18
# Data source: Ministry of Civil Affairs of the People's Republic of China
# (http://www.mca.gov.cn/article/sj/xzqh/2018/)

# This code use BaiduMap API to retrieve coordinates for prefectures in China 
# and then calculate the road distances of all prefecture pairs

# import libraries
import requests
import pandas as pd
import numpy as np

In [ ]:
# retrieve list of prefectures and store in 'pref_list'(omitted)

In [ ]:
### Retrieve coordinates of prefectures ###
# create a df to store coordinates
pref_co = pd.DataFrame(index = np.arange(len(pref_list)),
                       columns = ["prefecture","lat","lon","coord"])
pref_co["prefecture"] = pd.Series(pref_list)

# loop through prefecture list
for i in range(len(pref_co.index)):
    # request for the city's coordinates 
    prefname = pref_co.prefecture[i]
    c_endpoint = "http://api.map.baidu.com/geocoder/v2/?"
    c_params = {"address": prefname,
                "output": "json",
                "ak": "******"}

    get_coord = requests.get(c_endpoint, params=c_params)
    
    # store the coordinates in df
    result = get_coord.json()
    lat = result['result']['location']['lat']
    lon = result['result']['location']['lng']
    coord = str(lat)[:9]+","+str(lon)[:9]
    
    pref_co.lat.iat[i] = lat
    pref_co.lon.iat[i] = lon
    pref_co.coord.iat[i] = coord

In [30]:
# check
pref_co.head()

,prefecture,lat,lon
0,重庆市,29.544606,106.530635
1,保定市,38.886565,115.494810
2,石家庄市,38.048958,114.522082
3,成都市,30.679943,104.067923
4,邢台市,37.069531,114.520487


In [4]:
### Calculate distance between prefectures ###
# create a combination list of prefecture pairs
import itertools
pair_list = []
for subset in itertools.combinations(pref_list,2):
    pair_list.append(subset)

In [38]:
# df to store distances 
index = np.arange(len(pair_list))
columns = ['OriCity','DesCity']
pref_pairs = pd.DataFrame(index = index ,columns = columns)

# as pandas is slow with loops, use lists to store the data instead
OriCity = [None] * len(pair_list)
DesCity = [None] * len(pair_list)

for i in range(len(pair_list)):
    OriCity[i] = pair_list[i][0]
    DesCity[i] = pair_list[i][1]
    
pref_pairs["OriCity"] = OriCity
pref_pairs["DesCity"] = DesCity

# merge with coordinates df
pref_pairs = pd.merge(pref_pairs, pref_co[["prefecture","coord"]],how="left",
                       left_on="OriCity",right_on="prefecture")    
pref_pairs = pd.merge(pref_pairs, pref_co[["prefecture","coord"]],how="left",
                       left_on="DesCity",right_on="prefecture")  
pref_pairs = pref_pairs.drop(columns=["prefecture_x","prefecture_y"])

pref_pairs.columns = ['OriCity','DesCity','OriCoord','DesCoord']
pref_pairs.head()

,OriCity,DesCity,OriCoord,DesCoord
0,重庆市,保定市,"29.544606,106.53063","38.886564,115.49481"
1,重庆市,石家庄市,"29.544606,106.53063","38.048958,114.52208"
2,重庆市,成都市,"29.544606,106.53063","30.679942,104.06792"
3,重庆市,邢台市,"29.544606,106.53063","37.069531,114.52048"
4,重庆市,赣州市,"29.544606,106.53063","25.845295,114.93590"


In [10]:
# Retrieve distance by BaiduMap RouteMatrix API 
# limit of API is 30,000 calls/day

for i in range(len(Distance)):
    origin = pref_pairs.OriCoord[i]
    destination = pref_pairs.DesCoord[i]
            
    try:
        d_endpoint = "https://api.map.baidu.com/direction/v1/routematrix?"
        d_params ={"origins":origin,
                    "destinations":destination,
                    "output":"json",
                    "ak":"*******"}
        get_dist = requests.get(d_endpoint, params=d_params)
        
        # retrieve distances from the results
        result = get_dist.json()  
        
        if result['status'] == 0:
            Distance[i] = result['result']['elements'][0]['distance']['value']
            print(str(i) + result['message'])
        else:
            pass
    except:
        pass   
    
            

56950ok
56951ok
56952ok


In [127]:
# OPTION II: TEST with Direction API(slower than RouteMatrix)
# for i in range(55,100):  
#     lat = pref_pairs.OriCoord[i].split(",")[0][:8]
#     lon = pref_pairs.OriCoord[i].split(",")[1][:9]    
#     origin = str(lat)+","+str(lon)
    
#     dlat = pref_pairs.DesCoord[i].split(",")[0][:8]
#     dlon = pref_pairs.DesCoord[i].split(",")[1][:9] 
#     destination = str(dlat)+","+str(dlon)    
            
#     d_endpoint = "http://api.map.baidu.com/direction/v2/driving?"
#     d_params ={"origin":origin,
#                 "destination":destination,
#                 "output":"json",
#                 "ak":"********"}
#     get_dist = requests.get(d_endpoint, params=d_params)
        
#     # retrieve distances from the results
#     result = get_dist.json()
    
#     print(str(i)+";"+result['message'])
    
#     Distance[i] = result['result']['routes'][0]['distance']

# pref_pairs.head() 

In [11]:
# add the column to dataframe
pref_pairs["Distance"] = Distance
pref_pairs.tail()

,OriCity,DesCity,OriCoord,DesCoord,Distance
56948,三沙市,嘉峪关市,"12.464712,113.75535","39.802397,98.281634",0.0
56949,三沙市,中山市,"12.464712,113.75535","22.545177,113.42206",0.0
56950,儋州市,嘉峪关市,"19.574787,109.33458","39.802397,98.281634",3353008.0
56951,儋州市,中山市,"19.574787,109.33458","22.545177,113.42206",697260.0
56952,嘉峪关市,中山市,"39.802397,98.281634","22.545177,113.42206",3073535.0


In [44]:
### Calculate Distance Within Prefectures ###
# create a new df "pref_pairs_r" to store the within prefecture distances
index = np.arange(len(pref_list))
columns = ['OriCity','DesCity']
pref_pairs_r = pd.DataFrame(index = index ,columns = columns)

pref_pairs_r["OriCity"] = pref_list
pref_pairs_r["DesCity"] = pref_list

pref_pairs_r = pd.merge(pref_pairs_r, pref_co[["prefecture","coord"]],how="left",
                       left_on="OriCity",right_on="prefecture")    
pref_pairs_r = pd.merge(pref_pairs_r, pref_co[["prefecture","coord"]],how="left",
                       left_on="DesCity",right_on="prefecture")  
pref_pairs_r = pref_pairs_r.drop(columns=["prefecture_x","prefecture_y"])

pref_pairs_r.columns = ['OriCity','DesCity','OriCoord','DesCoord']
pref_pairs_r.head(1)

,OriCity,DesCity,OriCoord,DesCoord
0,重庆市,重庆市,"29.544606,106.53063","29.544606,106.53063"


In [12]:
# Calculate the average of distances between counties within a prefecture

# drop province and prefecture rows in division df
data_r = data.dropna(subset = ["县级"])

# list of prefectures and county combinations
pref_c = []
county_1 = []
county_2 = []

# loop through the prefecture list
for pref in pref_list:
    # subset of counties in the prefecture
    pref_sub = data_r.loc[data_r.地级 == pref]
    clist = pref_sub.县级.tolist()
    
    # get the combination of all counties
    for subset in itertools.combinations(clist,2):
        pref_c.append(pref)
        county_1.append(subset[0])
        county_2.append(subset[1])
    
# create a df of county combinations
c_pairs = pd.DataFrame(index = range(len(pref_c)), columns=["Pref","County_1","County_2"])
c_pairs["Pref"] = pref_c
c_pairs["County_1"] = county_1
c_pairs["County_2"] = county_2

c_pairs.head(1)

,Pref,County_1,County_2
0,重庆市,渝中区,万州区


In [17]:
# METHOD II(continue)
# list to store distance values
Distance = [np.nan]*len(c_pairs.index)

# loop through df    
for i in range(0,len(Distance)):
    origin = c_pairs.County_1[i]
    destination = c_pairs.County_2[i]
    
    # calculate the road distance between counties
    try:
        d_endpoint = "https://api.map.baidu.com/direction/v1/routematrix?"
        d_params ={"origins":origin,
                    "destinations":destination,
                    "output":"json",
                    "ak":"*******"}
        get_dist = requests.get(d_endpoint, params=d_params)
        
        # retrieve distances from the results
        result = get_dist.json()  
        
        if result['status'] == 0:
            Distance[i] = result['result']['elements'][0]['distance']['value']
            print(str(i) + result['message'])
        else:
            pass
    except:
        pass   
    
c_pairs["Distance"] = Distance
c_pairs.head(1)

0ok
1ok
2ok
4ok
5ok
6ok
7ok
8ok
9ok
10ok
11ok
12ok
13ok
14ok
15ok
16ok
17ok
18ok
19ok
20ok
21ok
22ok
23ok
24ok
25ok
26ok
27ok
28ok
29ok
30ok
31ok
32ok
33ok
34ok
35ok
36ok
37ok
38ok
40ok
41ok
42ok
43ok
44ok
45ok
46ok
47ok
48ok
49ok
51ok
52ok
53ok
54ok
55ok
56ok
57ok
58ok
59ok
60ok
61ok
62ok
63ok
64ok
65ok
66ok
67ok
68ok
69ok
70ok
71ok
72ok
73ok
75ok
76ok
77ok
78ok
79ok
80ok
81ok
82ok
83ok
84ok
85ok
86ok
87ok
88ok
89ok
90ok
91ok
92ok
93ok
94ok
95ok
96ok
97ok
98ok
99ok
100ok
101ok
102ok
103ok
104ok
105ok
106ok
107ok
109ok
110ok
111ok
112ok
113ok
114ok
115ok
116ok
117ok
118ok
119ok
120ok
121ok
122ok
123ok
124ok
125ok
126ok
127ok
128ok
129ok
130ok
131ok
132ok
133ok
134ok
135ok
136ok
137ok
138ok
139ok
140ok
141ok
175ok
176ok
177ok
178ok
179ok
180ok
181ok
182ok
183ok
184ok
185ok
186ok
187ok
188ok
189ok
190ok
191ok
192ok
193ok
194ok
195ok
196ok
197ok
198ok
199ok
200ok
201ok
202ok
203ok
204ok
205ok
206ok
207ok
208ok
209ok
210ok
211ok
212ok
213ok
214ok
215ok
216ok
217ok
218ok
219ok
220ok
221ok
2

1380ok
1381ok
1382ok
1383ok
1384ok
1385ok
1386ok
1387ok
1388ok
1389ok
1390ok
1391ok
1392ok
1393ok
1394ok
1395ok
1396ok
1397ok
1398ok
1399ok
1418ok
1419ok
1420ok
1421ok
1422ok
1423ok
1424ok
1425ok
1426ok
1427ok
1428ok
1429ok
1430ok
1431ok
1432ok
1433ok
1434ok
1435ok
1436ok
1437ok
1438ok
1439ok
1440ok
1441ok
1442ok
1443ok
1444ok
1445ok
1446ok
1447ok
1448ok
1449ok
1450ok
1451ok
1452ok
1453ok
1454ok
1455ok
1456ok
1457ok
1458ok
1459ok
1460ok
1461ok
1462ok
1463ok
1464ok
1465ok
1466ok
1467ok
1468ok
1469ok
1470ok
1471ok
1472ok
1473ok
1474ok
1475ok
1476ok
1477ok
1478ok
1479ok
1480ok
1481ok
1482ok
1483ok
1484ok
1485ok
1486ok
1487ok
1488ok
1489ok
1490ok
1491ok
1492ok
1493ok
1494ok
1495ok
1496ok
1497ok
1498ok
1499ok
1500ok
1501ok
1502ok
1503ok
1504ok
1505ok
1506ok
1507ok
1508ok
1509ok
1510ok
1511ok
1512ok
1513ok
1514ok
1515ok
1516ok
1517ok
1518ok
1519ok
1520ok
1521ok
1522ok
1523ok
1524ok
1525ok
1526ok
1527ok
1528ok
1529ok
1530ok
1531ok
1532ok
1533ok
1534ok
1535ok
1536ok
1537ok
1538ok
1539ok
1540ok

2569ok
2570ok
2571ok
2572ok
2573ok
2574ok
2575ok
2576ok
2577ok
2578ok
2579ok
2580ok
2581ok
2582ok
2583ok
2584ok
2585ok
2586ok
2587ok
2588ok
2589ok
2590ok
2591ok
2592ok
2593ok
2594ok
2595ok
2596ok
2597ok
2598ok
2599ok
2600ok
2601ok
2602ok
2603ok
2604ok
2605ok
2606ok
2607ok
2608ok
2609ok
2610ok
2611ok
2612ok
2613ok
2614ok
2615ok
2616ok
2617ok
2618ok
2619ok
2620ok
2621ok
2622ok
2623ok
2624ok
2625ok
2626ok
2627ok
2628ok
2629ok
2630ok
2631ok
2632ok
2633ok
2634ok
2635ok
2636ok
2637ok
2638ok
2639ok
2640ok
2641ok
2642ok
2643ok
2644ok
2645ok
2646ok
2647ok
2648ok
2649ok
2650ok
2651ok
2652ok
2653ok
2654ok
2655ok
2656ok
2657ok
2658ok
2659ok
2660ok
2661ok
2662ok
2663ok
2664ok
2665ok
2666ok
2667ok
2668ok
2669ok
2670ok
2671ok
2672ok
2673ok
2674ok
2675ok
2676ok
2677ok
2678ok
2679ok
2680ok
2681ok
2682ok
2683ok
2684ok
2685ok
2686ok
2687ok
2688ok
2689ok
2690ok
2691ok
2692ok
2693ok
2694ok
2695ok
2696ok
2697ok
2698ok
2699ok
2700ok
2701ok
2702ok
2703ok
2704ok
2705ok
2706ok
2707ok
2708ok
2709ok
2710ok
2711ok

3771ok
3772ok
3773ok
3774ok
3775ok
3776ok
3777ok
3778ok
3779ok
3780ok
3781ok
3782ok
3783ok
3784ok
3785ok
3786ok
3787ok
3788ok
3789ok
3790ok
3791ok
3792ok
3793ok
3794ok
3795ok
3796ok
3797ok
3798ok
3799ok
3800ok
3801ok
3802ok
3803ok
3804ok
3805ok
3806ok
3807ok
3808ok
3809ok
3810ok
3811ok
3812ok
3813ok
3814ok
3815ok
3816ok
3817ok
3818ok
3819ok
3820ok
3821ok
3822ok
3823ok
3824ok
3825ok
3826ok
3827ok
3828ok
3829ok
3830ok
3831ok
3832ok
3833ok
3834ok
3835ok
3836ok
3837ok
3838ok
3839ok
3840ok
3841ok
3842ok
3843ok
3844ok
3845ok
3846ok
3847ok
3848ok
3849ok
3850ok
3851ok
3852ok
3853ok
3854ok
3855ok
3856ok
3857ok
3858ok
3859ok
3860ok
3861ok
3862ok
3863ok
3864ok
3865ok
3866ok
3867ok
3868ok
3869ok
3870ok
3871ok
3872ok
3873ok
3874ok
3875ok
3876ok
3877ok
3878ok
3879ok
3880ok
3881ok
3882ok
3883ok
3884ok
3885ok
3886ok
3887ok
3888ok
3889ok
3890ok
3891ok
3892ok
3893ok
3894ok
3895ok
3896ok
3897ok
3898ok
3899ok
3900ok
3901ok
3902ok
3903ok
3904ok
3905ok
3906ok
3907ok
3908ok
3909ok
3910ok
3911ok
3912ok
3913ok

5004ok
5005ok
5006ok
5018ok
5019ok
5020ok
5021ok
5022ok
5023ok
5024ok
5025ok
5026ok
5027ok
5028ok
5029ok
5030ok
5031ok
5032ok
5033ok
5034ok
5035ok
5036ok
5037ok
5038ok
5039ok
5040ok
5041ok
5042ok
5043ok
5044ok
5045ok
5046ok
5047ok
5048ok
5049ok
5050ok
5051ok
5052ok
5053ok
5054ok
5055ok
5056ok
5057ok
5058ok
5059ok
5060ok
5061ok
5062ok
5063ok
5064ok
5065ok
5066ok
5067ok
5068ok
5069ok
5070ok
5071ok
5072ok
5096ok
5097ok
5098ok
5099ok
5100ok
5101ok
5102ok
5103ok
5104ok
5105ok
5106ok
5107ok
5108ok
5109ok
5110ok
5111ok
5112ok
5113ok
5114ok
5115ok
5116ok
5117ok
5118ok
5119ok
5120ok
5121ok
5122ok
5123ok
5124ok
5125ok
5126ok
5127ok
5128ok
5129ok
5130ok
5131ok
5132ok
5133ok
5134ok
5135ok
5136ok
5137ok
5138ok
5139ok
5140ok
5141ok
5142ok
5143ok
5144ok
5145ok
5146ok
5147ok
5148ok
5149ok
5150ok
5151ok
5152ok
5153ok
5154ok
5155ok
5156ok
5157ok
5158ok
5159ok
5160ok
5161ok
5162ok
5163ok
5164ok
5165ok
5166ok
5167ok
5168ok
5169ok
5170ok
5171ok
5172ok
5173ok
5174ok
5175ok
5176ok
5177ok
5178ok
5179ok
5180ok

...

9881ok
9882ok
9883ok
9884ok
9885ok
9886ok
9887ok
9888ok
9889ok
9890ok
9891ok
9892ok
9893ok
9894ok
9895ok
9896ok
9897ok
9898ok
9899ok
9900ok
9901ok
9902ok
9903ok
9904ok
9905ok
9906ok
9907ok
9908ok
9909ok
9910ok
9911ok
9912ok
9913ok
9914ok
9915ok
9916ok
9917ok
9918ok
9919ok
9920ok
9921ok
9922ok
9923ok
9924ok
9925ok
9926ok
9927ok
9928ok
9929ok
9930ok
9931ok
9932ok
9933ok
9934ok
9935ok
9936ok
9937ok
9938ok
9939ok
9940ok
9941ok
9942ok
9943ok
9944ok
9945ok
9946ok
9947ok
9948ok
9949ok
9950ok
9951ok
9952ok
9953ok
9954ok
9955ok
9956ok
9957ok
9958ok
9959ok
9960ok
9961ok
9962ok
9963ok
9964ok
9965ok
9966ok
9967ok
9968ok
9969ok
9970ok
9971ok
9972ok
9973ok
9974ok
9975ok
9976ok
9977ok
9978ok
9979ok
9980ok
9981ok
9982ok
9983ok
9984ok
9985ok
9986ok
9987ok
9988ok
9989ok
9990ok
9991ok
9992ok
9993ok
9994ok
9995ok
9996ok
9997ok
9998ok
9999ok
10000ok
10001ok
10002ok
10003ok
10004ok
10005ok
10006ok
10007ok
10008ok
10009ok
10010ok
10011ok
10012ok
10013ok
10014ok
10015ok
10016ok
10017ok
10018ok
10019ok
10020ok

10927ok
10928ok
10929ok
10930ok
10931ok
10932ok
10933ok
10934ok
10935ok
10936ok
10937ok
10938ok
10939ok
10940ok
10941ok
10942ok
10943ok
10944ok
10945ok
10946ok
10947ok
10948ok
10949ok
10950ok
10951ok
10952ok
10953ok
10954ok
10955ok
10956ok
10957ok
10958ok
10959ok
10960ok
10961ok
10962ok
10963ok
10964ok
10965ok
10966ok
10967ok
10968ok
10969ok
10970ok
10971ok
10972ok
10973ok
10974ok
10975ok
10976ok
10977ok
10978ok
10979ok
10980ok
10981ok
10982ok
10983ok
10984ok
10985ok
10986ok
10987ok
10988ok
10989ok
10990ok
10991ok
10992ok
10993ok
10994ok
10995ok
10996ok
10997ok
10998ok
10999ok
11000ok
11001ok
11002ok
11003ok
11004ok
11005ok
11006ok
11007ok
11008ok
11009ok
11010ok
11011ok
11012ok
11013ok
11014ok
11015ok
11016ok
11017ok
11018ok
11019ok
11020ok
11021ok
11022ok
11023ok
11024ok
11025ok
11026ok
11027ok
11028ok
11029ok
11030ok
11031ok
11032ok
11033ok
11034ok
11035ok
11036ok
11037ok
11038ok
11039ok
11040ok
11041ok
11042ok
11043ok
11044ok
11045ok
11046ok
11047ok
11048ok
11049ok
11050ok
11051ok


11960ok
11961ok
11962ok
11963ok
11964ok
11965ok
11966ok
11967ok
11968ok
11969ok
11970ok
11971ok
11972ok
11973ok
11974ok
11975ok
11976ok
11977ok
11978ok
11979ok
11980ok
11981ok
11982ok
11983ok
11984ok
11985ok
11986ok
11987ok
11988ok
11989ok
11990ok
11991ok
11992ok
11993ok
11994ok
11995ok
11996ok
11997ok
11998ok
11999ok
12000ok
12001ok
12002ok
12003ok
12004ok
12005ok
12006ok
12007ok
12008ok
12009ok
12010ok
12011ok
12012ok
12013ok
12014ok
12015ok
12016ok
12017ok
12018ok
12019ok
12020ok
12021ok
12022ok
12023ok
12024ok
12025ok
12026ok
12027ok
12028ok
12029ok
12030ok
12031ok
12032ok
12033ok
12034ok
12035ok
12036ok
12037ok
12038ok
12039ok
12040ok
12041ok
12042ok
12043ok
12044ok
12045ok
12046ok
12047ok
12048ok
12049ok
12050ok
12051ok
12052ok
12053ok
12054ok
12055ok
12056ok
12057ok
12058ok
12059ok
12060ok
12061ok
12062ok
12063ok
12064ok
12065ok
12066ok
12067ok
12068ok
12069ok
12070ok
12071ok
12072ok
12073ok
12074ok
12075ok
12076ok
12077ok
12078ok
12079ok
12080ok
12081ok
12082ok
12083ok
12084ok


13012ok
13013ok
13014ok
13015ok
13016ok
13017ok
13018ok
13019ok
13020ok
13021ok
13022ok
13023ok
13024ok
13025ok
13026ok
13027ok
13028ok
13029ok
13030ok
13031ok
13032ok
13033ok
13034ok
13035ok
13036ok
13037ok
13038ok
13039ok
13040ok
13041ok
13042ok
13043ok
13044ok
13045ok
13046ok
13047ok
13048ok
13049ok
13050ok
13051ok
13052ok
13053ok
13054ok
13055ok
13056ok
13057ok
13058ok
13059ok
13060ok
13061ok
13062ok
13063ok
13064ok
13065ok
13066ok
13067ok
13068ok
13069ok
13070ok
13071ok
13072ok
13073ok
13074ok
13075ok
13076ok
13077ok
13078ok
13079ok
13080ok
13081ok
13082ok
13083ok
13084ok
13085ok
13086ok
13087ok
13088ok
13089ok
13090ok
13091ok
13092ok
13093ok
13094ok
13095ok
13096ok
13097ok
13098ok
13099ok
13100ok
13101ok
13102ok
13103ok
13104ok
13105ok
13106ok
13107ok
13108ok
13109ok
13110ok
13111ok
13112ok
13113ok
13114ok
13115ok
13116ok
13117ok
13118ok
13119ok
13120ok
13121ok
13122ok
13123ok
13124ok
13125ok
13126ok
13127ok
13128ok
13129ok
13130ok
13131ok
13132ok
13133ok
13134ok
13135ok
13136ok


,Pref,County_1,County_2,Distance
0,重庆市,渝中区,万州区,263844.0


In [24]:
# METHOD II(continue)
# list to store distance values
Distance = [np.nan]*len(pref_pairs_r.index)
i = 0

# loop through df
for row in pref_pairs_r.itertuples():
    pref = row.OriCity
    
    # subset of counties in the prefecture
    pref_sub = c_pairs.loc[c_pairs.Pref == pref]
    
    # calculate the mean distance within each prefecture
    dist_avr = pref_sub.Distance.mean()
    Distance[i] = dist_avr
    
    i += 1
    
pref_pairs_r["Distance"] = Distance
pref_pairs_r.head(1)

,OriCity,DesCity,OriCoord,DesCoord,Distance
0,重庆市,重庆市,"29.544606,106.53063","29.544606,106.53063",226018.015038


In [78]:
# concat the dataframes
pref_pairs_rr = pd.concat([pref_pairs, pref_pairs_r])

In [7]:
# save the dataframes to files
pref_co.to_csv("data/CH_prefecture_coordinate.csv", index=False, encoding="utf_8_sig")
pref_pairs_rrr.to_csv("data/CH_prefecture_distance.csv", index=False, encoding="utf_8_sig")